# RNN pada Teks

Pada modul sebelumnya, kita telah membahas bagaimana memprediksi data sekuensial. Data dapat berwujud apa saja dan RNN merupakan sebuah bentuk model yang mendukung pemrosesan data sekuensial. Kali ini kita akan mencoba RNN pada bentuk data sekuensial lainnya yaitu teks.

### Library yang di Import
- `collections` digunakan untuk menyediakan beberapa container datatypes tambahan
- `re` digunakan untuk operasi regular expression (regex) pada python
- `d2l` digunakan untuk mengimport library yang dibutuhkan dalam modul ini yang bersumber dari buku [d2l.ai](https://d2l.ai/)

In [1]:
import collections
import re
from d2l import torch as d2l
import random

### Memuat Dataset

Pada modul ini, dataset yang digunakan bersumber dari sebuah buku [The Time Machine, oleh H.G. Wells](http://www.gutenberg.org/ebooks/35). Dataset yang akan digunakan berjumlah 30000 kata dan merupakan sebagian kecil dari keseluruhan kata yang ada pada buku tersebut. Untuk menyederhanakan operasi modul ini, kita akan mengabaikan tanda baca dan huruf besar.

Fungsi `read_time_machine()` akan memuat dataset dari buku tersebut menjadi sebuah array yang berisi kata-kata dari tiap baris. Fungsi tersebut juga akan mengabaikan huruf besar dan kecil serta tanda baca.

In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()

Dataset akan dipecah menjadi beberapa baris yang akan disimpan dalam array `lines`. Kita akan coba memeriksa panjang array `lines` untuk mengetahui jumlah baris yang ada dan kita akan mencetak beberapa contoh baris dari dataset.

In [3]:
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[random.randint(0,3000)])

# text lines: 3221
the time machine by h g wells
the fact is that insensibly the absolute strangeness of everything


### Tokenisasi

Tokenisasi merupakan fungsi yang akan menerima input berupa `lines` dan memecah masukan tersebut menjadi token. Token adalah unit dasar dari sebuah teks berupa string kata atau karakter.

In [4]:
def tokenize(lines, token='word'):  #@save
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

tokens = tokenize(lines)

'''
Mencetak contoh token
'''

for i in range(100,105):
    print(tokens[i])

['were', 'three', 'dimensional', 'representations', 'of', 'his', 'four', 'dimensioned']
['being', 'which', 'is', 'a', 'fixed', 'and', 'unalterable', 'thing']
[]
['scientific', 'people', 'proceeded', 'the', 'time', 'traveller', 'after', 'the', 'pause']
['required', 'for', 'the', 'proper', 'assimilation', 'of', 'this', 'know', 'very', 'well', 'that']


### Dictionary

Token berupa string atas kata maupun karakter pun masih belum memadai untuk dijadikan masukan bagi sebuah model. Perlu diingat bahwa dalam deep learning, model menerima masukan berupa vektor yang berisi nilai numerik sehingga kita harus membuat sebuah kamus kata (dictionary) yang menyatakan setiap token yang ada. Kamus kata ini akan menjadi sebuah array yang berisi setiap token yang ada pada dataset. Kamus kata atau dictionary juga kerap disebut sebagai vocabulary

Kamus kata akan berisi daftar kata (tokens) dan frekuensi kemunculannya. Kemudian, setiap kata akan diberi sebuah index numerik yang unik bergantung pada frekuensi kemunculannya, daftar dari kata ini disebut dengan corpus. Tokens yang jarang muncul tidak akan disertakan dalam dictionary supaya mengurangi kompleksitas. Token yang tidak ada dalam corpus tidak akan dijadikan masukan model dan akan dipetakan dengan `<unk>`.

Kamus kata ini akan dibuat dalam bentuk kelas

In [5]:

class KamusKata:
    
    def __init__(self,
                 tokens=None,
                 min_freq=0,
                 reserved_tokens=None
                 ):
        
        if tokens is None: # jika token kosong, maka token akan dijadikan list kosong
            tokens = []
            
        if reserved_tokens is None: # jika reserved_tokens kosong, maka reserved_tokens akan dijadikan list kosong
            reserved_tokens = []
            
        counter = count_corpus(tokens) # menghitung jumlah kata pada token
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True) # mengurutkan berdasarkan frekuensi
        
        self.idx_to_token = ['<unk>'] + reserved_tokens # apabila ada unknown token, maka indeksnya akan dijadikan 0
        
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)} # apabila ada unknown token, maka indeksnya akan dijadikan 0
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1
            
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens): # mengambil item dari token
        if not isinstance(tokens,(list,tuple)): # jika tokens bukan list atau tuple
            return self.token_to_idx.get(tokens, self.unk) # maka akan mengembalikan indeks unknown
        return [self.token_to_idx.get(token, self.unk) for token in tokens] # mengambil item dari indeks
    
    def to_tokens(self, indices): # mengubah indeks menjadi token
        if not isinstance(indices, (list, tuple)): # jika indices bukan list atau tuple
            return self.idx_to_token[indices] # maka akan mengembalikan token
        return [self.idx_to_token[idx] for idx in indices] # mengambil item dari indeks
    
    @property
    def unk(self):
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs
        
def count_corpus(tokens): 
        if len(tokens) == 0 or isinstance(tokens[0], list): # jika tokens kosong atau tokens pada indeks 0 adalah list:
            tokens = [token for line in tokens for token in line] # maka tokens akan dijadikan array 1 dimensi ??? [TODO]
        return collections.Counter(tokens) # menghitung jumlah kata

Penjelasan: Ketika sebuah Objek dibuat, maka akan terdapat beberapa atribut seperti:
- `tokens`: berisi daftar kata yang ada pada corpus
- `min_freq`: nilai minimum frekuensi dari kata yang akan dijadikan kamus kata
- `reserved_tokens`: berisi daftar token yang tidak akan dijadikan kamus kata

Silahkan lihat contoh penggunaan dictionary di bawah ini

### Contoh Penggunaan KamusKata

In [6]:
contoh_tokens = [['nama','saya','budi'],['nama','dia','amir'],['jessica','teman','amir'],['amir','siapa?'],['????'],[],['-']]

vocab = KamusKata(contoh_tokens)
print(vocab.token_to_idx)

{'<unk>': 0, 'amir': 1, 'nama': 2, 'saya': 3, 'budi': 4, 'dia': 5, 'jessica': 6, 'teman': 7, 'siapa?': 8, '????': 9, '-': 10}


Apabila `min_freq=2`, maka hanya kata yang muncul minimal 2 kali yang akan dijadikan dictionary

In [7]:
vocab = KamusKata(contoh_tokens, min_freq=2)
print(vocab.token_to_idx)
print(vocab.idx_to_token)

{'<unk>': 0, 'amir': 1, 'nama': 2}
['<unk>', 'amir', 'nama']


In [8]:
print(vocab['nama'])   # mencari indeks dari 'nama'
print(vocab['susilo']) # mencari indeks dari 'susilo' -> Nilai kembalian pasti menjadi 0 atau <unk> karena 'Susilo' tidak ada dalam corpus

2
0


## Menggunakan KamusKata
Sekarang mari kita gunakan `tokens` dan kita masukkan ke kelas KamusKata

In [9]:
vocab = KamusKata(tokens)
print(list(vocab.token_to_idx.items())[:10]) # menampilkan 10 token teratas dan dijadikan tuple -> ('tokennya', indeksnya)

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [10]:
for i in range (8,11):                      # mencetak corpus ke 8 sampai ke 10
    print(f'Kata', tokens[i])
    print(f'Indeks', vocab[tokens[i]])

Kata ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
Indeks [1, 19, 71, 16, 37, 11, 115, 42, 680, 6, 586, 4, 108]
Kata ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
Indeks [7, 1420, 5, 2185, 587, 6, 126, 25, 330, 127, 439, 3]
Kata ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
Indeks [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## Menggunakan Seluruh Fungsi Diatas

Fungsi `load_corpus_time_machine` akan melakukan:
- Membaca dataset dari buku tersebut
- Membuat `tokens` berdasarkan dataset
- Membuat `dictionary` berdasarkan `tokens`, terdiri atas karakter
- Membuat corpus berdasarkan setiap line dari tokens

In [12]:
def load_corpus_time_machine(max_tokens=-1):
    
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = KamusKata(tokens)
    
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
print(len(corpus), len(vocab))

print(corpus[:10])
print(vocab.token_freqs[:10])

170580 28
[3, 9, 2, 1, 3, 5, 13, 2, 1, 13]
[(' ', 29927), ('e', 17838), ('t', 13515), ('a', 11704), ('i', 10138), ('n', 9917), ('o', 9758), ('s', 8486), ('h', 8257), ('r', 7674)]


---
# Implementasi RNN menggunakan Teks

### Mengimpor library yang dibutuhkan

In [13]:
import math
import torch
from torch import nn
from torch.nn import functional as F

### Mempersiapkan dataset

In [14]:
batch_size = 32
num_steps = 35
corpus, vocab = d2l.load_data_time_machine(batch_size, num_steps)

### Inisiasi Parameter Model
Pada modul ini, model RNN akan diinisiasi secara manual tanpa menggunakan layer nn dari torch. Berikut penjelasan dari setiap layer yang digunakan:

<div>
<center>
<img src="../assets/rnn.jpeg", width="500">

Sumber: [d2l.ai](https://d2l.ai/)
</div>

- W adalah Weight: nilai bobot dari tiap neuron
- b adalah Bias: nilai bias dari tiap neuron
- X adalah Input: Input yang digunakan pada modul ini berupa teks
- h adalah Hidden State: nilai hidden state dari tiap neuron yang dipengaruhi berdasarkan hidden state sebelumnya
- q adalah Output: Output dari tiap neuron yang dipengaruhi berdasarkan hidden state saat ini dan input

Contoh:
- W_xh = Weight dari input ke hidden state
- W_hh = Weight dari hidden state ke hidden state
dan seterusnya

In [39]:
def get_params(vocab_size, hidden_size, device):
    input_size = output_size = vocab_size
    
    # Hidden Layer
    W_xh    = torch.randn((input_size, hidden_size), device=device)
    W_hh    = torch.randn((hidden_size, hidden_size), device=device)
    b_h     = torch.zeros(hidden_size, device=device)
    
    # Output Layer
    W_hq    = torch.randn((hidden_size, output_size), device=device)
    b_q     = torch.zeros(output_size, device=device)
    
    # Mengaktifkan semua gradient
    
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params
    

False


In [ ]:
class RNNModelScratch:
    def __init__(self,
                 vocab_size,
                 hidden_size,
                 ):
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.input_size = vocab_size
        self.output_size = vocab_size
        
         # Hidden Layer
        self.W_xh    = torch.randn((self.input_size, self.hidden_size))
        self.W_hh    = torch.randn((self.hidden_size,self.hidden_size))
        self.b_h     = torch.zeros(self.hidden_size)
        
        # Output Layer
        self.W_hq    = torch.randn((self.hidden_size, self.output_size))
        self.b_q     = torch.zeros(self.output_size)
        
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)